In [1]:
import os
import sys
paths = {
    'helpers': '/home/spadela/mids-w210-capstone'
}
for k,v in paths.items():
    sys.path.insert(0, v)
    
import requests
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

from bs4 import BeautifulSoup

### Set Up Mongo Connection
from db import dbToolsMongo
db = dbToolsMongo()

import time
import urllib.parse

pd.set_option('display.max_columns', 500)

%reload_ext autoreload
%autoreload 2

In [3]:
db.db.collection_names()

['icorating_people_details',
 'deadcoin_ico',
 'icorating_people',
 'foo',
 'sp_overlap_analysis',
 'sp_price_features',
 'sp_cryptocompare',
 'icostats_ico_price',
 '414_ico_names',
 'crypto_compare_ico',
 'icotracker_ico',
 'ico_subreddit',
 'ico_social_media_stats',
 'reddit_summary',
 'icorating_ratings',
 'sp_price_history']

In [4]:
db.toDF('icotracker_ico')

,ICO_JOIN_KEY,_id,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper
0,patron,5b1c8cf448d6215496c4d2c0,Ethereum,Start Bonus: 35% - Raised: 0,NaN,https://patron-ico.io/,A decentralized platform which shares economie...,"[fa, fa-close]",NaN,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",PATRON,https://www.linkedin.com/in/atsushi-hisatsumi/,Atsushi Hisatsumi,"{'Telegram': 'https://t.me/patronicoen', 'Face...",no_reddit,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...
1,cryptonvc,5b1c8cf448d6215496c4d2c1,Ethereum,Start Bonus: 30% - Raised: 0,NaN,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,"[fa, fa-close]",NaN,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,https://www.linkedin.com/in/jaceknowak2/,Jacek Nowak,"{'Github': 'https://github.com/CryptonVC', 'Te...",no_reddit,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf
2,sosr,5b1c8cf548d6215496c4d2c2,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,"[fa, fa-close]",NaN,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,https://icotracker.net/www.linkedin.com/in/cha...,Zhen Lin,{'Telegram': 'https://t.me/sosrcoin'},no_reddit,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...
3,dynatiq,5b1c8ef048d6215496c4d2c4,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,"[fa, fa-close]",NaN,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,https://www.linkedin.com/in/abdaluzza/,Abd al Uzza,"{'Blog': 'https://medium.com/@dynatiq', 'Teleg...",no_reddit,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf
4,3ccoin,5b1c8ef048d6215496c4d2c5,Waves,Start Bonus: 50% - Raised: 0,NaN,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,"[fa, fa-close]",NaN,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3cCoin,https://www.facebook.com/asb-service,Alexey Butorin,{'Youtube': 'https://www.youtube.com/channel/U...,no_reddit,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...
5,squadertokensale,5b1c8f2148d6215496c4d2c6,Ethereum,Start Bonus: 100% - Raised: 0,NaN,https://tokensale-squader.com/,A payment token for premium features in the ap...,"[fa, fa-close]",NaN,29/04/2018 00:00:00 UTC - 10/06/2018,"ICO: 8,000,000,000 SQC of 8,000,000,000",Squader Token Sale,NaN,-,"{'Blog': 'https://medium.com/@squadcoin', 'Tel...",no_reddit,https://icotracker.net/project/squader_token_sale,https://tokensale-squader.com/docs/Squadcoin-w...
6,smarterthancryptoico,5b1c8f2148d6215496c4d2c7,Ethereum,Start Bonus: 10% - Raised: 0,NaN,https://www.smarterthancrypto.com/,A cryptocurrency-only tokenized portfolio,"[fa, fa-close]",NaN,13/05/2018 08:00:00 UTC - 10/06/2018,"ICO: 87,000,000 STC of 100,000,000",Smarter Than Crypto (ICO),NaN,Oliver Prock,{'Reddit': 'https://www.reddit.com/user/Smarte...,not_a_sr,https://icotracker.net/project/smarter_than_cr...,https://www.smarterthancrypto.com/doc/Smarter_...
7,cellblocks,5b1c8f2148d6215496c4d2c8,Ethereum,Start Bonus: 0% - Raised: 0,NaN,https://cellblocks.io/,A digital solution to an outdated and corrupt ...,"[fa, fa-close]",NaN,01/04/2018 10:00:00 UTC - 10/06/2018,"ICO: 187,500,000 CLBK of 250,000,000",CellBlocks,https://www.linkedin.com/in/jonascbrown/,Jonas Brown,{'Reddit': 'https://www.reddit.com/user/cellbl...,not_a_sr,https://icotracker.net/project/cellblocks,https://cellblocks.io/whitepaper.pdf
8,cryptics,5b1c8f2148d6215496c4d2c9,Ethereum,Start Bonus: 72% - Raised: 0,NaN,http://cryptics.tech/,An AI-based public crypto analytics platform,"[fa, fa-close]",NaN,10/05/2018 12:00:00 UT

In [26]:
col = 'reddit_summary'
result = db[col].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
df1 = pd.DataFrame(data)
df1.head()

,Activity,ICO,Num_Comments,Total_Posts,Total_Score,_id
0,0,CatnipFun,0,1,1,5b31c9363fcd250bd56ddf58
1,0,chronobase,0,1,1,5b31c9363fcd250bd56ddf59
2,0,deckswap,0,1,1,5b31c9363fcd250bd56ddf5a
3,0,SmartCityICO,0,1,1,5b31c9363fcd250bd56ddf5b
4,0,BlindCoin,0,1,2,5b31c9363fcd250bd56ddf5c


In [27]:
result = db['icotracker_ico'].find({})

data = []
for i,row in enumerate(result):
    data.append(row)
    if i > 25:
        break
df1 = pd.DataFrame(data)[['company_url','description','ico_dates','ico_sold_status','name','owner_name','owner_linkedin_profile','tracker_url','whitepaper','status']]
df1.head()

,company_url,description,ico_dates,ico_sold_status,name,owner_name,owner_linkedin_profile,tracker_url,whitepaper,status
0,https://patron-ico.io/,A decentralized platform which shares economie...,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",PATRON,Atsushi Hisatsumi,https://www.linkedin.com/in/atsushi-hisatsumi/,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...,no_reddit
1,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,Jacek Nowak,https://www.linkedin.com/in/jaceknowak2/,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,no_reddit
2,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,Zhen Lin,https://icotracker.net/www.linkedin.com/in/cha...,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,no_reddit
3,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,Abd al Uzza,https://www.linkedin.com/in/abdaluzza/,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,no_reddit
4,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3cCoin,Alexey Butorin,https://www.facebook.com/asb-service,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...,no_reddit


In [47]:
import datetime
dates = []
text = "09/05/2018 00:00:00 UTC - 09/06/2018"
for txt in text.split(' '):
    try:
        valid_date = datetime.datetime.strptime(txt.strip(),'%d/%m/%Y')
        dates.append(datetime.datetime.strftime(valid_date,'%d/%m/%Y'))
    except ValueError as e:
        # Not a date, fail silently...
        pass
dates

['09/05/2018', '09/06/2018']

In [40]:
datetime.datetime.strptime('09/05/2018','%d/%m/%Y')

datetime.datetime(2018, 5, 9, 0, 0)

In [48]:
import datetime
def format_ico_dates(dates_text):
    dates = []
    for txt in dates_text.split(' '):
        try:
            valid_date = datetime.datetime.strptime(txt.strip(),'%d/%m/%Y')
            dates.append(datetime.datetime.strftime(valid_date,'%d/%m/%Y'))
        except ValueError as e:
            # Not a date, fail silently...
            pass
    if len(dates) == 1:
        dates = dates*2
    return ' - '.join(dates)

df1['key_dates'] = df1.apply(lambda x: format_ico_dates(x['ico_dates']),axis=1)
df1.head()

,company_url,description,ico_dates,ico_sold_status,name,owner_name,owner_linkedin_profile,tracker_url,whitepaper,status,key_dates
0,https://patron-ico.io/,A decentralized platform which shares economie...,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",PATRON,Atsushi Hisatsumi,https://www.linkedin.com/in/atsushi-hisatsumi/,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...,no_reddit,09/05/2018 - 09/06/2018
1,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,Jacek Nowak,https://www.linkedin.com/in/jaceknowak2/,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,no_reddit,29/03/2018 - 09/06/2018
2,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,Zhen Lin,https://icotracker.net/www.linkedin.com/in/cha...,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,no_reddit,10/04/2018 - 09/06/2018
3,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,Abd al Uzza,https://www.linkedin.com/in/abdaluzza/,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,no_reddit,25/04/2018 - 09/06/2018
4,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3cCoin,Alexey Butorin,https://www.facebook.com/asb-service,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...,no_reddit,10/05/2018 - 10/06/2018


In [16]:
df1['name_url'] = df1.apply(lambda x: x['name'] + '|' + x['company_url'],axis=1)
df1['owner_url'] = df1.apply(lambda x: x['owner_name'] + '|' + str(x['owner_linkedin_profile']),axis=1)
df1.head()

,company_url,description,ico_dates,ico_sold_status,name,owner_name,owner_linkedin_profile,tracker_url,whitepaper,status,name_url,owner_url
0,https://patron-ico.io/,A decentralized platform which shares economie...,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",PATRON,Atsushi Hisatsumi,https://www.linkedin.com/in/atsushi-hisatsumi/,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...,no_reddit,PATRON|https://patron-ico.io/,Atsushi Hisatsumi|https://www.linkedin.com/in/...
1,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,Jacek Nowak,https://www.linkedin.com/in/jaceknowak2/,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,no_reddit,Crypton VC|http://crypton.vc/,Jacek Nowak|https://www.linkedin.com/in/jacekn...
2,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,Zhen Lin,https://icotracker.net/www.linkedin.com/in/cha...,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,no_reddit,SOSR|https://www.sosrcoin.io/en/index.html,Zhen Lin|https://icotracker.net/www.linkedin.c...
3,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,Abd al Uzza,https://www.linkedin.com/in/abdaluzza/,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,no_reddit,Dynatiq|https://dynatiq.com/,Abd al Uzza|https://www.linkedin.com/in/abdalu...
4,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3cCoin,Alexey Butorin,https://www.facebook.com/asb-service,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...,no_reddit,3cCoin|https://3ccoin.io/en/,Alexey Butorin|https://www.facebook.com/asb-se...


In [24]:
def make_link(label,url):
    return "<a href='{url}'>{label}</url>".format(label=label,url=url)

df1['links'] = df1.apply(lambda x: '<br/>'.join([make_link('URL',x['company_url']),make_link('Tracker',x['tracker_url']),make_link('Whitepaper',x['whitepaper'])]),axis=1)
df1.head()

,company_url,description,ico_dates,ico_sold_status,name,owner_name,owner_linkedin_profile,tracker_url,whitepaper,status,name_url,owner_url,links
0,https://patron-ico.io/,A decentralized platform which shares economie...,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",PATRON,Atsushi Hisatsumi,https://www.linkedin.com/in/atsushi-hisatsumi/,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...,no_reddit,PATRON|https://patron-ico.io/,Atsushi Hisatsumi|https://www.linkedin.com/in/...,<a href='https://patron-ico.io/'>URL</url><br/...
1,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,Jacek Nowak,https://www.linkedin.com/in/jaceknowak2/,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,no_reddit,Crypton VC|http://crypton.vc/,Jacek Nowak|https://www.linkedin.com/in/jacekn...,<a href='http://crypton.vc/'>URL</url><br/><a ...
2,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,Zhen Lin,https://icotracker.net/www.linkedin.com/in/cha...,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,no_reddit,SOSR|https://www.sosrcoin.io/en/index.html,Zhen Lin|https://icotracker.net/www.linkedin.c...,<a href='https://www.sosrcoin.io/en/index.html...
3,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,Abd al Uzza,https://www.linkedin.com/in/abdaluzza/,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,no_reddit,Dynatiq|https://dynatiq.com/,Abd al Uzza|https://www.linkedin.com/in/abdalu...,<a href='https://dynatiq.com/'>URL</url><br/><...
4,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3cCoin,Alexey Butorin,https://www.facebook.com/asb-service,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...,no_reddit,3cCoin|https://3ccoin.io/en/,Alexey Butorin|https://www.facebook.com/asb-se...,<a href='https://3ccoin.io/en/'>URL</url><br/>...


In [6]:
df1 = df1.fillna('').to_dict(orient='records')

AttributeError: 'list' object has no attribute 'head'